# Introduction

The PERCEIVE project seeks to proactively identify upcoming cybersecurity threats through textual similarity. Social network analysis, however, can be used to partition textual content, and hence serve as social-oriented word selection criteria, for defining corpus' documents. 

The subject of this notebook, the [Full Disclosure (FD) mailing list](http://seclists.org/fulldisclosure/) is a "public, vendor-neutral forum for detailed discussion of vulnerabilities and exploitation techniques, as well as tools, papers, news, and events of interest to the community."

## Problem Statement

Although cybersecurity email mailing lists provide a rich source to identify emerging concepts, it contains a large amount of content that is irrelevant to the project's purpose, including but not limited to:

 * conference invitations[[1](http://seclists.org/fulldisclosure/2017/Feb/6)]
 * vendor announcements[[2](http://seclists.org/fulldisclosure/2016/Dec/48)]
 * extensive conversations on security topics[[3](http://seclists.org/fulldisclosure/2004/Jul/1026)]
 * a significant amount of trolling[[4](http://seclists.org/fulldisclosure/2008/Apr/96)] 
 * and nonsense[[5](http://seclists.org/fulldisclosure/2009/Jul/289)] [[6](http://seclists.org/fulldisclosure/2004/Jul/796)].

As some of the irrelevant content can be strictly tied to it's producer, i.e., the **authors** of the **e-mail replies**, Social network analysis provides an interesting opportunity for the isolation of relevant discussion topics in order to **filter** non-related vulnerability content. 

## Network Definition

Earlier in this project, the FD email lists were developed into networks of edges and nodes, divided by year. The original csv files are available [online](https://mega.nz/#F!CUEByR5I!GY56GzTpYz68IlTqj4aQNQ!fR8jFLxL). The networks were imported into [Gephi](https://gephi.org/) to create a series of [visualization graphics](
https://mega.nz/#F!btdgFBID!hktkVrhZB5etOBBVLrgTrA).

In the FD graphs, nodes represent either documents (i.e., emails) or authors. Edges are directed, representing authorship and replies; edge weight indicates an increasing number of replies. In the Gephi graphics, blue nodes represent authors and red nodes represent documents. 


# Full Disclosure Exploration

## Considered Tools

### Gephi 

While Gephi has provided useful for data exploration, some difficulties arised. In particular, many of the core functions[[7](https://github.com/jaroslav-kuchar/Social-Network-Analysis/issues/2)] and plugins[[8](https://github.com/gephi/gephi/issues/1481)] once used for social network analysis are not compatible with current (0.9) software versions. Gephi specifications note that the 0.9.0 version (December 2015) "Removed [the] Clustering module"[[9](https://github.com/gephi/gephi/releases/tag/v0.9.0)] that these plugins relied upon.

### Python-igraph 

[Python-igraph](http://igraph.org/python/) (igraph hereafter) allows a user to import graphs in a variety of file formats, several of which match Gephi's export options. Brief tests demonstrated that the [GraphML](http://graphml.graphdrawing.org/) format was most successful for a workflow of transferring networks from Gephi to igraph. (Attempt to import [GML](https://en.wikipedia.org/wiki/Graph_Modelling_Language) and [Pajek](http://mrvar.fdv.uni-lj.si/pajek/) caused one instance of Python to hang; further investigation would be required to identify a cause.) The following examples use [GraphML files](https://mega.nz/#F!Dpdh2TjD!4Rd462mFXbdFn5Scs1WwUA) exported from Gephi.

## Network Definition 

In [14]:
# GraphML files may be imported into igraph with a single command (`Read_GraphML`) and then analyzed:
from igraph import Graph, summary
# import the GraphML files previously exported from Gephi
ml = Graph.Read_GraphML('data/2008.graphml')

ml['name'] = 'Full Disclosure network'
# summary reports the edges, nodes, and overall attributes in use. 
summary(ml)

IGRAPH D-W- 9793 6463 -- Full Disclosure network
+ attr: name (g), Color (v), Label1 (v), Label2 (v), b (v), g (v), id (v), label (v), nodeType (v), r (v), size (v), x (v), y (v), Edge Label (e), id (e), weight (e)


The summary command providing the output seen above is explained in the igraph documentation[[10](http://igraph.org/python/doc/igraph.summary'.GraphSummary-class.html)]. In this example, the four-character code "D-W-" indicates that the graph is directed and weighted. The graph has 984 vertices (nodes) and 758 edges. 

The attributes given in the summary ("name", "Color", etc.) are those for the graph (g), vertices (v), or edges (e).

In [4]:
# show the id attribute of the first 2 vertices
print('ID list of first two vertices in network:')
print(ml.vs[0:2]['id'])


ID list of first two vertices in network:
['andur matrix <andurmatrix () gmail com>', 'Adam Muntner <adam.muntner () quietmove com>']


Nodes can either be **authors** as shown above, or **e-mail threads** (in which case their identifier is the year, month and relative id for the given month).

In [5]:
# identify vertices with the highest degree and betweenness
# (This sample is borrowed directly from the tutorial!)

print('\nIDs of vertices with highest degree:', ml.vs.select(_degree = ml.maxdegree())['id'])


IDs of vertices with highest degree: ['security () mandriva com']


It is often common that higher degree nodes (those who post several e-mail replies in e-mail threads) are advisories, and do not really communicate. We can clearly see that the author above, by his e-mail, is clearly a research group in the mailing list, and those are more related to advisories per our manual observations. 

## Full Disclosure Visualization over Years 

The GraphML files from Gephi include the ids as _labels_, and so they are included by default when igraph creates visualizations. These may be eliminated by deleting the attribute.

Igraph handles visualization through _layouts_. Layouts are separate objects from the graphs themselves; multiple layouts can be created per graph.

Visual graphs are created via `plot` commands, however it demands too much time to render, and visualization is deferred to gephi and them edited on an image editor and assembled as a gif of 5s delay for easier visualization:

In [6]:
# import the GraphML files previously exported from Gephi
# delete label attributes to avoid visual clutter. In this example, the 'label'
# attribute is a duplicate of the 'ID' attribute so there is no need to save the label
#del ml.vs['label']

#layout = ml.layout_graphopt() # create a Fruchterman Reingold layout for the graph

#png = plot(ml, 'output/plot.png', layout=layout, bbox=(1000,1000), margin=100,
#           edge_curved=True)  # the bbox keyword argument defines the dimensions



The resulting graphic provides a visual overview of the FD 2016 network:

![An igraph-generated FD graph for 2016](https://github.com/sailuh/perceive/raw/master/Websites/Project/fulldisclosure_nooverlap_curved.gif)



We can observe that the network structure varies considerably across the years. This provides an opportunity to partition a network at a given year into several clusters and investigate if the visualized structure has any association to the textual content of a given **e-mail thread** discussion.

If such association exists, then we can leverage the **social network structure** in order to simplify the identification of emerging concepts through text alone. For instance, identifying a group of individuals have preference to certain subjects, or are spammers may become a trivial pre-processing stage before textual content is analyzed for emerging concepts.

We begin by considering 2 methods to more precisely define how to partition a network at any given year: 

 * Community Detection 
 * Betweness Centrality

# Community Detection

Some of the community measures do not support directed graphs, as seen in the warning message below, but the direction can be removed to compare the results of igraph's community funtions.

In [7]:
com = ml.community_leading_eigenvector(weights='weight', clusters=3)

C:\Anaconda3\lib\site-packages\igraph\__init__.py:1072: RuntimeWarning: This method was developed for undirected graphs at src\community.c:1565
  membership, _, q = GraphBase.community_leading_eigenvector(self, clusters, **kwds)


In [15]:
ml.to_undirected() # eliminate the directionality

com = ml.community_leading_eigenvector(clusters=3)
print('clustering attempt, leading eigenvector:')
summary(com)

clustering attempt, leading eigenvector:
Clustering with 9793 elements and 4708 clusters


A more detailed look at the eigenvector clusters follows in two parts. First, a listing of the overall cluster contents:

In [18]:
print(len(com), 'clusters.')
print('maximum size:', len(max(com, key=len)))
print('minimum size:', len(min(com, key=len)))

print('\nSummary of first 20 clusters:')
for i in range(19):
    etc = ''
    if len(com[i]) > 5:
        etc += '(and ' + str(len(com[i]) - 5) + ' more)'
    print('[{}] has size of {}. Vertices: {} {}'.format(i, len(com[i]), com[i][0:5], etc))

4708 clusters.
maximum size: 3468
minimum size: 1

Summary of first 20 clusters:
[0] has size of 2. Vertices: [0, 2438] 
[1] has size of 3468. Vertices: [1, 2, 4, 5, 6] (and 3463 more)
[2] has size of 14. Vertices: [3, 2442, 2760, 3090, 3420] (and 9 more)
[3] has size of 1. Vertices: [12] 
[4] has size of 3. Vertices: [16, 2448, 2991] 
[5] has size of 6. Vertices: [28, 9556, 9577, 9607, 9652] (and 1 more)
[6] has size of 2. Vertices: [37, 2462] 
[7] has size of 2. Vertices: [40, 2465] 
[8] has size of 256. Vertices: [41, 1188, 2466, 2473, 2629] (and 251 more)
[9] has size of 2. Vertices: [42, 2467] 
[10] has size of 6. Vertices: [43, 2468, 2540, 3605, 3906] (and 1 more)
[11] has size of 42. Vertices: [45, 3563, 3697, 3766, 7104] (and 37 more)
[12] has size of 2. Vertices: [48, 2474] 
[13] has size of 7. Vertices: [52, 2315, 7281, 7334, 8441] (and 2 more)
[14] has size of 4. Vertices: [53, 2480, 3767, 8200] 
[15] has size of 1. Vertices: [55] 
[16] has size of 1. Vertices: [56] 
[17] ha

Second, a report of the first 10 communities' node types and labels:

In [19]:
for i in range(10):
    print('COMMUNITY', i, '  ( total size:', len(com[i]), ')')
    for _ in com[i][0:10]:
        print('\t', ml.vs[_]['nodeType'], ':\t', ml.vs[_]['label'])
    if len(com[i]) > 10:
        print('\t... TRUNCATED LISTING')
    print()

COMMUNITY 0   ( total size: 2 )
	 author :	 andur matrix <andurmatrix () gmail com>
	 document :	 Re: [OOT] Thesis for master degree

COMMUNITY 1   ( total size: 3468 )
	 author :	 Adam Muntner <adam.muntner () quietmove com>
	 author :	 Denzity <denzity () gmail com>
	 author :	 Marcin Wielgoszewski <marcinw86 () gmail com>
	 author :	 Geo. <geoincidents () nls net>
	 author :	 coderman <coderman () gmail com>
	 author :	 reepex <reepex () gmail com>
	 author :	 Javor Ninov <drfrancky () securax org>
	 author :	 SilentRunner <silentrunner () hushmail com>
	 author :	 php0t <php0t () zorro hu>
	 author :	 Simon Smith <simon () snosoft com>
	... TRUNCATED LISTING

COMMUNITY 2   ( total size: 14 )
	 author :	 Matousec - Transparent security Research <research () matousec com>
	 document :	 Kerio Fake 'iphlpapi' DLL injection Vulnerability
	 document :	 Outpost Bypassing Self-Protection using file	links Vulnerability
	 document :	 Comodo Multiple insufficient argument validation of hooked

# Betweenness Centrality

Igraph betweenness measures, like the clustering algorithms, require undirected graphs. Betweenness is returned as a simple list of values.

In [20]:
bc = ml.betweenness()
bc.sort(reverse=True);

In [21]:
max_bc = max(bc)
bc_normalized = [x / max_bc for x in bc]

for idx,val in enumerate(bc_normalized[0:17]):
    print('Node',idx,':',"{:.0%}".format(bc_normalized[idx]))

Node 0 : 100%
Node 1 : 47%
Node 2 : 36%
Node 3 : 34%
Node 4 : 33%
Node 5 : 32%
Node 6 : 32%
Node 7 : 32%
Node 8 : 31%
Node 9 : 31%
Node 10 : 30%
Node 11 : 30%
Node 12 : 29%
Node 13 : 29%
Node 14 : 27%
Node 15 : 22%
Node 16 : 18%


We can observe that the betweeness centrality quickly drops past the 16th node, which may indicate nodes that often discuss different subjects for contributing e-mail discussions for different "community of threads". 

# Exporting and utilizing analyzed graph data

Igraph allows users to export graph files for use in other software such as Gephi.

In [39]:
# add betweenness centrality and clustering info to original graph
bc = ml.betweenness()
for idx, node in enumerate(ml.vs):
    ml.vs[idx]['betweenness'] = bc[idx]

for idx, c in enumerate(com):
    for _ in c:
        ml.vs[_]['cluster'] = idx

ml.save('data/fd2008_out.graphml', format='graphml') # export graph


After opening this 2008 graph in Gephi, we can manipulate colors and sizes of nodes to reflect attributes identified in igraph. Adjusting colors based on clustering, and node size based on betweenness centrality, provides a visual overview of the newly-identified features:

![FD2008 graph from Gephi](data/2008_clusters_betweenness.png)

We can observe thousands of small nodes with minimal betweenness value (over 8000 nodes in the graph have a betweenness of zero), and a handful of large nodes with higher values. The clusters are colored via cluster partition; one large cluster colored purple accounts for more than a third of the overall graph.

Manual analysis of the large (high-betweenness) clusters indicates that these nodes represent users[[1](https://www.bing.com/search?q=site%3Aseclists.org+Valdis.Kletnieks+%28%29+vt+edu)][[2](https://www.bing.com/search?q=site%3Aseclists.org+3APA3A+%3C3APA3A+%28%29+SECURITY+NNOV+RU)] who are extremely active on the mailing list in multiple threads, partially confirming our hypothesis.



In [ ]:
# TO DO: manual analysis of clusters;
#        code to automate the process for multiple years;
#        hypotheses for filtering/deleting clusters or nodes by attribute